### Метод отжига для задачи коммивояжера

In [ ]:
import numpy as np
import random
import time
import itertools
from typing import List, Tuple
import math

Генерация случайного графа

In [ ]:
def generate_random_graph(size: int, min_weight: float, max_weight: float) -> np.ndarray:
    graph = np.zeros((size, size))
    for i in range(size):
        for j in range(i + 1, size):
            weight = random.uniform(min_weight, max_weight)
            graph[i][j] = weight
            graph[j][i] = weight

    return graph

Вычисление стоимости пути в графе

In [ ]:
def path_cost(path: List[int], graph: np.ndarray) -> float:
    cost = 0.0
    n = len(path)
    for i in range(n - 1):
        cost += graph[path[i]][path[i+1]]
    cost += graph[path[-1]][path[0]]  # Замкнуть цикл
    return cost

Генерация соседнего решения, зависящая от температуры

In [ ]:
def generate_neighbor(current: List[int], temperature: float, initial_temp: float) -> List[int]:
    neighbor = current.copy()
    n = len(neighbor)

    # Нормализованная температура (0-1)
    normalized_temp = temperature / initial_temp

    # Чем выше температура, тем более радикальные изменения
    if normalized_temp > 0.5:
        # Высокая температура - большие изменения (перестановка двух случайных вершин)
        i, j = random.sample(range(n), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    elif normalized_temp > 0.2:
        # Средняя температура - средние изменения (переворот подпоследовательности)
        i, j = sorted(random.sample(range(n), 2))
        neighbor[i:j+1] = reversed(neighbor[i:j+1])
    else:
        # Низкая температура - маленькие изменения (перестановка соседних вершин)
        i = random.randint(0, n - 2)
        neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]

    return neighbor

Моделирование отжига

In [ ]:
def simulated_annealing(
    graph: np.ndarray,
    initial_temp: float,
    cooling_rate: float,
    min_temp: float,
    max_iterations: int) -> List[int]:

    n = graph.shape[0]
    current_path = list(range(n))
    random.shuffle(current_path)

    best_path = current_path.copy()
    current_cost = path_cost(current_path, graph)
    best_cost = current_cost

    temp = initial_temp
    iteration = 0

    while temp > min_temp and iteration < max_iterations:
        # print("Temp:", temp, iteration)
        neighbor = generate_neighbor(current_path, temp, initial_temp)
        neighbor_cost = path_cost(neighbor, graph)
        delta = neighbor_cost - current_cost

        if delta < 0 or math.exp(-delta / temp) > random.random():
            current_path = neighbor
            current_cost = neighbor_cost

            if current_cost < best_cost:
                best_path = current_path.copy()
                best_cost = current_cost
                print(f"Новое лучшее решение: {best_cost:.2f} (температура: {temp:.2f})")

        temp *= cooling_rate
        iteration += 1

    return best_path

Точное решение методом полного перебора (только для небольших графов)

In [ ]:
def brute_force(graph: np.ndarray) -> List[int]:
    n = graph.shape[0]
    best_path = list(range(n))
    best_cost = path_cost(best_path, graph)

    # Перебираем все перестановки
    for perm in itertools.permutations(range(1, n)):
        path = [0] + list(perm)
        cost = path_cost(path, graph)
        if cost < best_cost:
            best_cost = cost
            best_path = path

    return best_path

Запуск моделирования с заданным коэффициентом охлаждения

In [ ]:
def simulation(graph: np.ndarray, cooling_rate: float, exact_cost: float, compare_exact: bool):
    # Параметры алгоритма
    initial_temp = 10000
    min_temp = 1e-5
    max_iterations = 100000
    graph_size = graph.shape[0]

    print(f"Запуск имитации отжига для графа из {graph_size} вершин при коэффициенте охлаждения {cooling_rate}")
    start = time.time()

    result = simulated_annealing(graph, initial_temp, cooling_rate, min_temp, max_iterations)
    result_cost = path_cost(result, graph)

    end = time.time()
    duration = end - start

    print("\nРезультат имитации отжига:")
    print("Лучший путь:", result)
    print(f"Стоимость: {result_cost:.2f}")
    print(f"Время выполнения: {duration:.2f} секунд")

    if compare_exact:
        deviation = ((result_cost - exact_cost) / exact_cost * 100)
        print(f"Отклонение от оптимального решения: {deviation:.2f}%")

        if abs(result_cost - exact_cost) < 1e-6:
            print("Алгоритм нашел оптимальное решение!")
        else:
            print("Алгоритм нашел приближенное решение.")
    print("\n\n")

Сценарий тестирования

In [ ]:
# Генерация графа с 10 вершинами
graph_size = 10
graph = generate_random_graph(graph_size, 1.0, 50.0)

# Точное решение для сравнения (только для небольших графов)
exact_cost = 0
if graph_size < 11:
  exact_cost = path_cost(brute_force(graph), graph)

# Запуск симуляций с разными коэффициентами охлаждения
simulation(graph, 0.99, exact_cost, graph_size < 11)
simulation(graph, 0.995, exact_cost, graph_size < 11)
simulation(graph, 0.999, exact_cost, graph_size < 11)
simulation(graph, 0.9995, exact_cost, graph_size < 11)
simulation(graph, 0.99999, exact_cost, graph_size < 11)

Запуск имитации отжига для графа из 10 вершин при коэффициенте охлаждения 0.99
Новое лучшее решение: 219.79 (температура: 10000.00)
Новое лучшее решение: 214.92 (температура: 9227.45)
Новое лучшее решение: 209.32 (температура: 6689.72)
Новое лучшее решение: 175.46 (температура: 5989.56)
Новое лучшее решение: 170.40 (температура: 55.39)
Новое лучшее решение: 159.77 (температура: 37.80)
Новое лучшее решение: 157.26 (температура: 13.29)

Результат имитации отжига:
Лучший путь: [1, 4, 0, 2, 8, 6, 5, 3, 7, 9]
Стоимость: 157.26
Время выполнения: 0.01 секунд
Отклонение от оптимального решения: 21.86%
Алгоритм нашел приближенное решение.



Запуск имитации отжига для графа из 10 вершин при коэффициенте охлаждения 0.995
Новое лучшее решение: 167.25 (температура: 6305.56)
Новое лучшее решение: 158.02 (температура: 5562.89)
Новое лучшее решение: 155.04 (температура: 5398.08)
Новое лучшее решение: 152.97 (температура: 19.20)
Новое лучшее решение: 142.88 (температура: 5.71)

Результат имитации отжи